In [1]:
# Simulating dynamics of Argon atoms
# Starting with outlining general kinematics of particles

Week 1:
Play around with this system. Start by simulating the time evolution of a few particles in a periodic box, 
add the forces due to the Lennard-Jones potential. Check how the total energy of your system evolves over time.
It's easier to start with a 2D system, but plan to switch to 3D at a later stage.
Week 2:
Derive the expression of the kinetic energy in dimensionless units
Write a molecular dynamics code that uses dimensionless units and simulate a few atoms. 
Plot both kinetic, potential and total energy.
Boundary condition, minimal image convention

In [5]:
import numpy as np
from astropy import constants as const
from astropy import units as u

In [6]:
# Lennard-Jones potential 
def LJP(e, sig, r):
    '''Lennard-Jones potential formula
    Describes the potential of the system given a distance between two particles
    takes in epsilon (kB * Kelvin) and sigma (AA)
    function of magnitude distance r'''
    return 4*e*((sig/r)**12. - (sig/r)**6.)

def dUdr(e, sig, r):
    ''' Derivative of LJP WRT r
    given epsilon and sigma
    function of magnitude distance r'''
    return -24.*sig**6.*e*(2.*sig**6.*r**(-13.) - r**(-7.))

def force(derivU, x, r):
    '''Force from LJP potential
    derivU from function dUdr
    x is vector
    r is magnitude'''
    nablaU = derivU*(x/r)
    return -nablaU

# New velocity
def next_velocity(v_prev, m, F, t):
    '''Finding the next velocity value
    initial velocity v_init
    mass of object
    force F at position x_init
    timestep t
    Check units'''
    return v_prev + (1/m)*F*t

# New position given a timestep
def next_position(x_prev, v_prev, t):
    '''The next position value
    initial position x_init
    initial velocity v_init
    timestep t
    Make sure length and time units are the same'''
    return x_prev + v_prev*t

In [19]:
# Variables
# Argon
epsilon = const.k_B * 119.8*u.K
sigma = 3.405*u.AA
mass = 39.948*u.u

In [20]:
# The force at x is the negative gradient of the potential at x
# The gradient of the potential is the derivative of the potential WRT r times x/r
# where r is the magnitude of the distance and x is a vector

# Initial conditions
x_init = [30., 0, 0]*u.AA # initial position vector
r_init = np.sqrt(x_init[0]**2. + x_init[1]**2. + x_init[2]**2.) # magnitude
v_init = [1., 0, 0]*(u.AA/u.second) # initial velocity
h = 0.5*u.second
num_time = 10  # number of time steps

# U = LJP(epsilon, sigma, r)
F = force(dUdr(epsilon, sigma, r), x, r)


In [29]:
timesteps = np.zeros((6, num_time))  # 6 degrees of freedome: 3 position coordinates, 3 velocities
timesteps[:3, 0] = x_init
timesteps[3:, 0] = v_init
print(timesteps)
# Change in position
for t in range(1, num_time):
    v_next = next_velocity(timesteps[3:, t-1], mass, F, h)
    x_next = next_position(timesteps[:3, t-1], timesteps[3:, t-1], h)
    timesteps[:3, t] = x_next
    timesteps[3:, t] = v_next

# print(timesteps)

[[30.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


UnitsError: Can only apply 'add' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)